In [1]:
import gym
from env.WarthogEnv import WarthogEnv
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
from matplotlib import pyplot as plt
from stable_baselines3 import PPO

In [ ]:
#env = DummyVecEnv([lambda: WarthogEnv('unity_remote.txt')])
env = WarthogEnv('unity_remote.txt')

In [ ]:
model = PPO('MlpPolicy', env, verbose=1)

In [ ]:
model.learn(total_timesteps=1e7)

In [ ]:
x = []
y = []
env.reset()
for i in range(0,1000):
    action = [0.5,0.1]
    obs, reward, done, info = env.step(action)
    x.append(info[0][0])
    y.append(info[0][1])
    if done:
        print("resetting")
        env.reset()
    print(info)


In [ ]:
print(env.num_waypoints)

In [ ]:
plt.plot(x,y,'+r')
plt.ylim(-3, 9)
plt.xlim(9, 21)